#Zhen Lu 2018/07/15
#post-Processing of the sampled data, take conditional average with respect to
#the mixture fraction

In [1]:
import glob
import numpy as np
import pandas as pd

In [2]:
# pick the latest time automatically
calc_time=[]
for filename in glob.glob('surfaces/*'):
    calc_time.append(float(filename[9:]))
time=str(max(calc_time))

# extract variable and location names
var_names = []
loc_names = []
foldername = 'surfaces/{t}/'.format(t=time)
str_mean = 'Mean'
str_loc = '_z'
# variables
for filename in glob.glob('{0}*.raw'.format(foldername)):
    if str_mean not in filename and str_loc in filename:
        var_names.append(filename[len(foldername):filename.find(str_loc)])
var_names=list(set(var_names))
# must have mixture fraction Z
if 'Z' not in var_names:
    raise RuntimeError
for filename in glob.glob('{0}{1}{2}*.raw'.format(foldername,'Z',str_loc)):
    loc_names.append(filename[filename.find(str_loc)+len(str_loc):filename.find('.raw')])

In [3]:
num_bins_0 = 40
num_bins_1 = 40
Z_bins = np.concatenate((np.arange(0,0.2,0.2/float(num_bins_0)),
                         np.arange(0.2,1,0.8/float(num_bins_1)),
                         [1,]))
Z_mids = (Z_bins[1:] + Z_bins[:-1])/2

In [4]:
for loc in loc_names:
    # sample region
    r_limit = 0.04
    if int(loc) < 40:
        r_limit = 13.6 + (28-13.6)*float(loc)/40.
    # read coordinates
    filename = '{0}Z_z{1}.raw'.format(foldername,loc)
    data = np.genfromtxt(filename,skip_header=2,usecols=(0,1))
    # calculate radius
    r = np.sqrt(np.sum(np.square(data),axis=1))
    
    nrows = np.sum( r<r_limit )
    ncols = len(var_names)
    
    data_scat = np.zeros((nrows,ncols))
    data_scat_all = np.zeros((1,ncols))

    # loop over time
    for time in calc_time:
        foldername = 'surfaces/{}/'.format(time)
        
        for i, var in enumerate(var_names):
            filename = '{0}{1}_z{2}.raw'.format(foldername,var,loc)
            data = np.genfromtxt(filename,skip_header=2,usecols=(3,))
            # guarantee all value positive
            data[ data < 0 ] = 0
            data_scat[:,i] = data[ r < r_limit ]

        np.savetxt('{0}scat_z{1}.csv'.format(foldername,loc),
                   data_scat,
                   fmt='%15.6e',
                   delimiter=',',
                   header=','.join(var_names),
                   comments='')
        
        data_scat_all = np.concatenate((data_scat_all,data_scat))
    
    data_scat_all = np.delete(data_scat_all,0,0)
    
    df_tmp = pd.DataFrame(data_scat_all,columns=var_names)
    df = df_tmp.groupby(np.digitize(df_tmp['Z'],Z_bins,right=True)).mean()
    df.to_csv('cond_z{}.csv'.format(loc),
              index=False,
              float_format='%15.6e')